In [113]:
import numpy as np

n_states = 3
n_time = 5

def convert_transition_rate_to_probabilities(transition_rate, dt=1):
    transition_rate *= dt
    transition_matrix = transition_rate.copy()
    
    n_states = transition_matrix.shape[1]
    transition_matrix[:, np.arange(n_states), np.arange(n_states)] = 1
    
    norm = 1 + np.sum(transition_rate, axis=-1) - transition_rate[:, np.arange(n_states), np.arange(n_states)]
    
    return transition_matrix / norm[:, :, np.newaxis]
    
transition_rate = np.arange(n_states * n_states * (n_time - 1)).reshape((n_time - 1, n_states, n_states))
convert_transition_rate_to_probabilities(transition_rate, dt=1)
    

array([[[0.25      , 0.25      , 0.5       ],
        [0.33333333, 0.11111111, 0.55555556],
        [0.42857143, 0.5       , 0.07142857]],

       [[0.04545455, 0.45454545, 0.5       ],
        [0.44444444, 0.03703704, 0.51851852],
        [0.46875   , 0.5       , 0.03125   ]],

       [[0.025     , 0.475     , 0.5       ],
        [0.46666667, 0.02222222, 0.51111111],
        [0.48      , 0.5       , 0.02      ]],

       [[0.01724138, 0.48275862, 0.5       ],
        [0.47619048, 0.01587302, 0.50793651],
        [0.48529412, 0.5       , 0.01470588]]])

In [ ]:
def negative_log_likelihood(coefficients, design_matrix, joint_prob):
    transition_rate = np.exp(design_matrix @ coefficients)
    return -1.0 * (joint_prob * convert_transition_rate_to_probabilities(transition_rate, dt=1)).sum()

In [151]:
def transition_negative_log_likelihood(transition_rate, joint_prob, acausal_posterior, dt=1):
    """Negative log likelihood of transition matrix.

    Parameters
    ----------
    transition_rate : np.ndarray, shape (n_time - 1, n_states, n_states)
        The rate of transition from one state to another.
    joint_prob : np.ndarray, shape (n_time - 1, n_states, n_states)
        The joint probability of the current and next state.
    acausal_posterior : np.ndarray, shape (n_time, n_states)
        The acausal posterior probability of the current state.
    dt : int, optional
        Time step, by default 1

    Returns
    -------
    negative_log_likelihood : float
        The negative log likelihood of the transition matrix.
    """
    transition_rate *= dt
    numerator = (joint_prob * np.log(transition_rate)).sum(axis=-1) - (joint_prob[:, np.arange(n_states), np.arange(n_states)] * np.log(transition_rate[:, np.arange(n_states), np.arange(n_states)]))
    denominator = (acausal_posterior[:-1] * np.log1p(np.sum(transition_rate, axis=-1) - transition_rate[:, np.arange(n_states), np.arange(n_states)]))
    return -1.0 * (numerator - denominator[..., np.newaxis]).sum()

In [152]:
joint_prob = np.ones((n_time - 1, n_states, n_states))
acausal_posterior = np.ones((n_time, n_states))

transition_negative_log_likelihood(transition_rate, joint_prob, acausal_posterior, dt=1)

/var/folders/86/m147b4k17lddvs_xsw0mj2zw0000gn/T/ipykernel_97176/418189776.py:23: RuntimeWarning: divide by zero encountered in log
  return -1.0 * (joint_prob * np.log(transition_rate) - norm[..., np.newaxis]).sum()


inf

In [153]:
norm = (acausal_posterior[:-1] * np.log1p(np.sum(transition_rate, axis=-1) - transition_rate[:, np.arange(n_states), np.arange(n_states)]))
blah = (joint_prob * np.log(transition_rate) - norm[..., np.newaxis])

/var/folders/86/m147b4k17lddvs_xsw0mj2zw0000gn/T/ipykernel_97176/1478723949.py:2: RuntimeWarning: divide by zero encountered in log
  blah = (joint_prob * np.log(transition_rate) - norm[..., np.newaxis])


In [154]:
blah[0]

array([[       -inf, -1.38629436, -0.69314718],
       [-1.09861229, -0.81093022, -0.58778666],
       [-0.84729786, -0.69314718, -0.55961579]])

In [156]:
argh = np.zeros((n_time - 1, n_states))

for k in range(n_time - 1):
    for n in range(n_states):
        for m in range(n_states):
            if n != m:
                argh[k, n] += joint_prob[k, n, m] * np.log(transition_rate[k, n, m])
                
            argh[k, n] -= acausal_posterior[k, n] * np.log1p(transition_rate[k, n].sum() - )

SyntaxError: invalid syntax (3859807681.py, line 9)

/var/folders/86/m147b4k17lddvs_xsw0mj2zw0000gn/T/ipykernel_97176/1898782678.py:1: RuntimeWarning: divide by zero encountered in log
  (joint_prob * np.log(transition_rate)).sum(axis=-1) - (joint_prob[:, np.arange(n_states), np.arange(n_states)] *
/var/folders/86/m147b4k17lddvs_xsw0mj2zw0000gn/T/ipykernel_97176/1898782678.py:2: RuntimeWarning: divide by zero encountered in log
  np.log(transition_rate[:, np.arange(n_states), np.arange(n_states)]))
/var/folders/86/m147b4k17lddvs_xsw0mj2zw0000gn/T/ipykernel_97176/1898782678.py:1: RuntimeWarning: invalid value encountered in subtract
  (joint_prob * np.log(transition_rate)).sum(axis=-1) - (joint_prob[:, np.arange(n_states), np.arange(n_states)] *


array([[       nan, 2.7080502 , 3.73766962],
       [4.70048037, 5.12396398, 5.48063892],
       [5.94017125, 6.18001665, 6.39692966],
       [6.69950034, 6.86693328, 7.02286809]])

In [166]:
blah = np.arange(18).reshape((2, 3, 3))

np.diagonal(blah, axis1=1, axis2=2)

array([[ 0,  4,  8],
       [ 9, 13, 17]])

In [169]:
blah.sum(axis=-1) - np.diagonal(blah, axis1=1, axis2=2)

array([[ 3,  8, 13],
       [21, 26, 31]])

In [178]:
from scipy.special import xlogy, xlog1py

def transition_negative_log_likelihood(coefficients, design_matrix, joint_prob, acausal_posterior, dt=1):
    """Negative log likelihood of transition matrix.

    Parameters
    ----------
    coefficients : np.ndarray
        The coefficients of the linear model for the transition rate.
    design_matrix : np.ndarray, shape (n_time - 1, n_states, n_states, n_features)
        The design matrix representing the linear model for the transition rate.
    joint_prob : np.ndarray, shape (n_time - 1, n_states, n_states)
        The joint probability of the current and next state.
    acausal_posterior : np.ndarray, shape (n_time, n_states)
        The acausal posterior probability of the current state.
    dt : int, optional
        Time step, by default 1

    Returns
    -------
    negative_log_likelihood : float
        The negative log likelihood of the transition matrix.
    """
    transition_rate = np.exp(design_matrix @ coefficients) * dt
    return -1.0 * (
        np.sum(xlogy(joint_prob, transition_rate), axis=-1) -
        np.diagonal(xlogy(joint_prob, transition_rate), axis1=1, axis2=2) -
        xlog1py(acausal_posterior[:-1], np.sum(transition_rate, axis=-1) - np.diagonal(transition_rate, axis1=1, axis2=2))
    ).sum()



/var/folders/86/m147b4k17lddvs_xsw0mj2zw0000gn/T/ipykernel_97176/704593671.py:5: RuntimeWarning: invalid value encountered in subtract
  np.sum(xlogy(joint_prob, transition_rate), axis=-1) -


array([[       nan, 0.51082562, 1.09861229],
       [1.60943791, 1.82812711, 2.01490302],
       [2.2512918 , 2.37335416, 2.48490665],
       [2.63905733, 2.72379856, 2.80336038]])

In [179]:
transition_rate.shape

(4, 3, 3)

In [ ]:
# transitions estimated for transitions away from state
# n_state * (n_state - 1) * n_features transition filter parameters 
# stimulus of shape (n_time, n_features)
# design_matrix of shape (n_time, n_features) @ coefficients of shape (n_features,)
# transition_prob of shape (n_time, n_states, n_states)

# link function must be exp. Can this be a multinomial GLM with softmax



In [ ]:
# transition design matrix (n_time, n_coefficients, n_states, n_states)
# coefficients (n_coefficients, n_state * (n_state - 1)), each column corresponds to the off-digonal elements of the transition matrix
# estimate each row of the transition matrix independently (base case is self transition)
# Need to simulate simple base case where the transition matrix is stationary for half the time and then switches to another stationary matrix for the other half of the time
# 3 states

In [357]:
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt

# Set the random seed for reproducibility
np.random.seed(42)

# Parameters
num_states = 3
num_time_points = 1_000_000
sampling_rate = 250
true_rates = np.array([10.0, 20.0, 30.0]) / sampling_rate  # Constant rates for the Poisson emission distributions

# Define the stationary transition matrix
true_transition_matrix = np.array(
    [
     [0.9999, (1-0.9999)/2, (1-0.9999)/2],
     [0.01, 0.98, 0.01],
     [0.01, 0.01, 0.98]
     ]
)

# Define the initial probabilities for each state
true_initial_conditions = np.array([1.0, 0.0, 0.0])

# Define the Poisson HMM
model = hmm.PoissonHMM(n_components=num_states, random_state=42, init_params="", params="")
model.startprob_ = true_initial_conditions
model.transmat_ = true_transition_matrix
model.lambdas_ = true_rates.reshape(-1, 1)

# Simulate the HMM
observations, hidden_states = model.sample(num_time_points)

estimated_transition = np.zeros((num_states, num_states))
for prev_state in range(num_states):
    for cur_state in range(num_states):
        estimated_transition[prev_state, cur_state] = np.logical_and(
            hidden_states[:-1] == prev_state, hidden_states[1:] == cur_state).sum() / (hidden_states[:-1] == prev_state).sum()
        
np.diagonal(estimated_transition)

array([0.99991729, 0.98175626, 0.98145216])

In [358]:
from src.hmm import check_converged
from src.hmm3 import estimate_parameters_via_em
import scipy.stats


initial_conditions = true_initial_conditions.copy()
diag_prob = np.array([0.98, 0.98, 0.98])
transition_matrix = ((1 - diag_prob) / 2)[:, np.newaxis] * np.ones((num_states, num_states))
transition_matrix[np.diag_indices(num_states)] = diag_prob
initial_rates = true_rates.copy()
log_likelihood = scipy.stats.poisson.logpmf(observations, initial_rates[np.newaxis])

marginal_log_likelihoods = []
n_iter = 0
converged = False
max_iter = 50
tolerance=1e-4

while not converged and (n_iter < max_iter):
    (
        initial_conditions,
        transition_matrix,
        acausal_posterior,
        marginal_log_likelihood,
    ) = estimate_parameters_via_em(initial_conditions, log_likelihood, transition_matrix)
    
    rates = np.array(
        [np.average(observations.squeeze(), weights=acausal_posterior[:, state])
         for state in range(num_states)])
    log_likelihood = scipy.stats.poisson.logpmf(observations, rates[np.newaxis])

    # Stats
    print("Stats")
    n_iter += 1

    marginal_log_likelihoods.append(marginal_log_likelihood)
    if n_iter > 1:
        log_likelihood_change = (
            marginal_log_likelihoods[-1] - marginal_log_likelihoods[-2]
        )
        converged, _ = check_converged(
            marginal_log_likelihoods[-1], marginal_log_likelihoods[-2], tolerance
        )

        print(
            f"iteration {n_iter}, "
            f"likelihood: {marginal_log_likelihoods[-1]}, "
            f"change: {log_likelihood_change}"
        )
    else:
        print(f"iteration {n_iter}, " f"likelihood: {marginal_log_likelihoods[-1]}")


Stats
iteration 1, likelihood: -178682.27899745954
Stats
iteration 2, likelihood: -172068.27991221775, change: 6613.999085241783
Stats
iteration 3, likelihood: -172009.91340906662, change: 58.36650315113366
Stats
iteration 4, likelihood: -171981.37794274898, change: 28.535466317640385
Stats
iteration 5, likelihood: -171965.83882373004, change: 15.539119018940255


In [359]:
np.diagonal(transition_matrix)

array([0.98907156, 0.97725369, 0.96060461])

In [360]:
initial_conditions = true_initial_conditions.copy()
diag_prob = np.array([0.98, 0.98, 0.98])
transition_matrix = ((1 - diag_prob) / 2)[:, np.newaxis] * np.ones((num_states, num_states))
transition_matrix[np.diag_indices(num_states)] = diag_prob
initial_rates = true_rates.copy()

model2 = hmm.PoissonHMM(n_components=num_states, random_state=42, init_params="", params="stl")
model2.startprob_ = initial_conditions
model2.transmat_ = transition_matrix
model2.lambdas_ = initial_rates.reshape(-1, 1)

model2.fit(observations)

PoissonHMM(init_params='', n_components=3,
           random_state=RandomState(MT19937) at 0x17EAB6F40)

In [361]:
np.diagonal(model2.transmat_)

array([0.98916073, 0.97722909, 0.96002886])

In [362]:
model2.monitor_.history

deque([-178682.27900102452,
       -172068.2799162289,
       -172009.9134081912,
       -171981.3779422958,
       -171965.83882000495,
       -171956.69376626197,
       -171950.9816733937,
       -171947.23761563213,
       -171944.68041641908,
       -171942.8681272074])

In [436]:
# coefficients (n_coefficients, n_state * (n_state - 1)), each column corresponds to the off-digonal elements of the transition matrix
# estimate each row of the transition matrix independently (base case is self transition)

n_coefficients = 2
n_states = 4
transition_coefficients = np.zeros(((n_coefficients, n_states * (n_states - 1))))

# all off diagonal indicies
transition_coefficient_index = np.stack(np.unravel_index(np.arange(n_states * n_states), (n_states, n_states)), axis=1)
is_diagonal = transition_coefficient_index[:, 0] == transition_coefficient_index[:, 1]
transition_coefficient_index = transition_coefficient_index[~is_diagonal]


from_state = 2
is_from_state = transition_coefficient_index[:, 0] == from_state
transition_coefficient_linear_index = np.arange(transition_coefficients.shape[1])
from_state_index = transition_coefficient_linear_index[is_from_state]
transition_coefficients[:, from_state_index], transition_coefficient_index[is_from_state, 1]


(array([[0., 0., 0.],
        [0., 0., 0.]]),
 array([0, 1, 3]))

In [441]:
from_state = 0
to_state = 2


array([False,  True, False, False, False, False, False, False, False,
       False, False, False])

In [442]:
from_state = 2

is_from_state = transition_coefficient_index[:, 0] == from_state
transition_coefficient_linear_index = np.arange(transition_coefficients.shape[1])
from_state_index = transition_coefficient_linear_index[is_from_state]
blah = 0.0
expected_joint_prob = np.zeros((n_time, n_states, n_states))


np.exp(transition_design_matrix @ transition_coefficients[:, is_state])

for to_state in range(n_states):
    is_state = np.logical_and(transition_coefficient_index[:, 0] == from_state,
                              transition_coefficient_index[:, 1] == to_state)
    if from_state != to_state:
        
        blah += xlogy(expected_joint_prob[:, from_state, to_state],
                      transition_design_matrix @ transition_coefficients[:, is_state])
    else:
        blah += xlog1py(expected_joint_prob[:, from_state, to_state],
                        norm)

NameError: name 'expected_joint_prob' is not defined

In [444]:
transition_coefficients[:, is_state].T.shape

(1, 2)

In [468]:
n_time = 20

transition_design_matrix = np.zeros((n_time, n_coefficients))

transition_design_matrix @ transition_coefficients[:, is_state].shape

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [469]:
(transition_design_matrix @ transition_coefficients).shape

(20, 12)

In [476]:
transition_matrix = np.ones((n_time, n_states, n_states))

dt = 1

transition_matrix[:, transition_coefficient_index[:, 0], transition_coefficient_index[:, 1]] = (transition_design_matrix @ transition_coefficients) * dt
transition_matrix

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0.,

array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3])

In [ ]:
from scipy.special import xlogy, xlog1py


def transition_negative_log_likelihood(
    transition_coefficients,
    transition_design_matrix,
    joint_prob,
    transition_coefficient_index,
    acausal_posterior,
    dt=1,
):
    """Negative log likelihood of transition matrix.

    Parameters
    ----------
    transition_coefficients : np.ndarray, shape (n_cofficients, n_states * (n_states - 1))
        The coefficients of the linear model for the transition rate.
    transition_design_matrix : np.ndarray, shape (n_time - 1, n_coefficients)
        The design matrix representing the linear model for the transition rate.
    joint_prob : np.ndarray, shape (n_time - 1, n_states, n_states)
        The joint probability of the current and next state.
    acausal_posterior : np.ndarray, shape (n_time, n_states)
        The acausal posterior probability of the current state.
    dt : int, optional
        Time step, by default 1

    Returns
    -------
    negative_log_likelihood : float
        The negative log likelihood of the transition matrix.
    """
    transition_rate = np.zeros_like(joint_prob)
    transition_rate[
        :, transition_coefficient_index[:, 0], transition_coefficient_index[:, 1]
    ] = (np.exp(transition_design_matrix @ transition_coefficients) * dt)

    return (
        -1.0
        * (
            np.sum(xlogy(joint_prob, transition_rate), axis=-1)
            - np.diagonal(xlogy(joint_prob, transition_rate), axis1=1, axis2=2)
            - xlog1py(
                acausal_posterior[:-1],
                np.sum(transition_rate, axis=-1)
                - np.diagonal(transition_rate, axis1=1, axis2=2),
            )
        ).sum()
    )


In [ ]:
from scipy.special import xlogy, xlog1py


def transition_negative_log_likelihood(
    transition_coefficients,
    transition_design_matrix,
    joint_prob,
    transition_coefficient_index,
    acausal_posterior,
    dt=1,
):
    """Negative log likelihood of transition matrix.

    Parameters
    ----------
    transition_coefficients : np.ndarray, shape (n_coefficients, n_states - 1)
        The row coefficients of the linear model for the transition rate.
    transition_design_matrix : np.ndarray, shape (n_time - 1, n_coefficients)
        The design matrix representing the linear model for the transition rate.
    joint_prob : np.ndarray, shape (n_time - 1, n_to_states)
        The joint probability of the current and next state.
    acausal_posterior : np.ndarray, shape (n_time, n_states)
        The acausal posterior probability of the current state.
    dt : int, optional
        Time step, by default 1

    Returns
    -------
    negative_log_likelihood : float
        The negative log likelihood of the transition matrix.
    """
    transition_rate = np.zeros_like(joint_prob)
    transition_rate[
        :, transition_coefficient_index[:, 0], transition_coefficient_index[:, 1]
    ] = (np.exp(transition_design_matrix @ transition_coefficients) * dt)

    return (
        -1.0
        * (
            np.sum(xlogy(joint_prob, transition_rate), axis=-1)
            - np.diagonal(xlogy(joint_prob, transition_rate), axis1=1, axis2=2)
            - xlog1py(
                acausal_posterior[:-1],
                np.sum(transition_rate, axis=-1)
                - np.diagonal(transition_rate, axis1=1, axis2=2),
            )
        ).sum()
    )
